## Get text embeddings

## Hardware check

In [1]:
# gpu check
! nvidia-smi

/bin/bash: nvidia-smi: command not found


In [9]:
! python --version

Python 3.9.7


In [2]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [3]:
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# # from tensorflow.keras.models import Model
# from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
# from sklearn.utils.class_weight import compute_class_weight
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
# import tensorflow.keras.backend as K
# from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
# from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files
import gc

2024-01-16 10:52:00.156014: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 10:52:01.081223: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 10:52:20.236565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Version Check

In [6]:
import tensorflow as tf
print(tf. __version__)

pickle.format_version

2.12.0


'4.0'

## Load Data

In [4]:
# path to glove data
data_path = 'Data/glove_kws_reserved.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [5]:
pred_window = 2  # hours
obs_windows = range(20, 124, 4)

# Remove train, val patients
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(train_sub)]
data = data.loc[~data.SUBJECT_ID.isin(valid_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(train_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(valid_sub)]

data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]


def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d


static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1

# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']
            ].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_texts_ip = []
fore_inds = []


def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask


def pad(x):
    return x+[0]*(fore_max_len-len(x))


for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour >= w) & (data.hour <= w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg(
        {'value': 'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg(
        {'vind_value': list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)
    obs_data = data.loc[(data.hour < w) & (data.hour >= w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg(
        {'vind': list, 'hour': list, 'value': list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
        
        
    fore_inds.append(np.array(list(obs_data.ts_ind)))

    #     matrix = list(obs_data.value)
    #     obs_strings = []
    #     for l in matrix:
    #         string_list = []
    #         for value in l:
    #             if isinstance(value, str):
    #                 string_list.append(value)
    #         obs_strings.append(string_list)
    #     del matrix
    #     fore_texts_ip.append(np.array(obs_strings))
    # del data
    values = list(obs_data.value)
    texts = []
    for value in values:
        texts.append(np.array([item for item in value if isinstance(item, str)]))
    fore_texts_ip.append(np.array(texts))
    

fore_texts_ip = np.concatenate(fore_texts_ip, axis=0)

  0%|          | 0/26 [00:00<?, ?it/s]/scratch/slurm_tmpdir/job_23053689/ipykernel_93498/3544100277.py:95: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(texts))
  4%|▍         | 1/26 [00:04<01:51,  4.46s/it]/scratch/slurm_tmpdir/job_23053689/ipykernel_93498/3544100277.py:95: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(texts))
  8%|▊         | 2/26 [00:08<01:44,  4.37s/it]/scratch/slurm_tmpdir/job_23053689/ipykernel_93498/3544100277.py:95: VisibleDeprecationWarning: Creating an ndarray from

In [6]:
len(fore_texts_ip)

131920

In [11]:
test_texts = []
for text in tqdm(fore_texts_ip):
    test_texts.append(' '.join(text))

test_texts = np.array(test_texts)

NameError: name 'fore_texts_ip' is not defined

In [10]:
del fore_texts_ip
np.savetxt('Exp_GloVe/texts_ip/forecasting_test_ip.txt', test_texts, fmt='%s')